# Self-Attention with Trainable Weights

In [12]:
# Step 1: Import necessary libraries
import torch
import torch.nn.functional as F  # Used for softmax

#### Below are the input embeddings where each word/friend has some numerical meaning

In [14]:
# Step 2: Define the input (4 friends, each with 3 features)
# Each row represents one friend (or word in a sentence)
friends = torch.tensor([
    [0.2, 0.4, 0.6],   # Alice
    [0.1, 0.3, 0.5],   # Bob
    [0.9, 0.1, 0.2],   # Charlie
    [0.7, 0.5, 0.9]    # Daisy
])

#### We create three trainable weight matrices for Query, Key, and Value.

- W_Q: Learns how to ask good questions
- W_K: Learns how to describe what each word is about
- W_V: Learns how to give useful information

These are learned during training, its like the model figuring out how to be a better listener over time.

In [16]:
# Step 3: Create trainable weights for Query (Q), Key (K), and Value (V)
# These help the model learn how to focus on relevant information
W_Q = torch.nn.Linear(3, 3, bias=False)
W_K = torch.nn.Linear(3, 3, bias=False)
W_V = torch.nn.Linear(3, 3, bias=False)

#### We apply the weights to the inputs:

- Q: What each word is asking
- K: What each word is offering
- V: The content each word holds

For example, when the model sees "it," the Query helps it ask, "Who are you referring to?"

In [18]:
# Step 4: Generate Q, K, V by applying the weights to the inputs
Q = W_Q(friends)
K = W_K(friends)
V = W_V(friends)

#### We compute how well each query matches each key.

- Q . K^T: Measures similarity between words
- Divide by √(dimension) to stabilize gradients (this is called scaled dot-product)

This gives us raw attention scores stating, how much one word should attend to another.

In [20]:
# Step 5: Calculate attention scores (similarity between Q and K)
# Then scale them to keep values stable
scores = torch.matmul(Q, K.T) / (Q.size(-1) ** 0.5)

#### We convert raw scores into probabilities using softmax
So now each word knows:
"I’ll listen 70% to ‘Alice’, 20% to ‘Bob’, 10% to others.”

In [26]:
# Step 6: Convert scores to attention weights using softmax
attention_weights = F.softmax(scores, dim=1)

#### We take the weighted sum of Values.

Each word combines other's content, based on how much attention it paid, this is the final contextual embedding.

In [30]:
# Step 7: Use attention weights to get the final output (context-aware representations)
output = torch.matmul(attention_weights, V)

# Step 8: Print the result
print("What the model understood:\n", output)

What the model understood:
 tensor([[ 0.5762, -0.3826, -0.2915],
        [ 0.5757, -0.3829, -0.2920],
        [ 0.5800, -0.3864, -0.2945],
        [ 0.5799, -0.3821, -0.2896]], grad_fn=<MmBackward0>)


#### What does this output mean?
- Each row is the updated version of a word after it has "listened" to the others.
- The model now understands each word not just on its own, but in the context of the whole sentence.
- These vectors will be used in the next layers to help the model make smarter decisions, like predicting the next word or answering questions.
